<a href="https://colab.research.google.com/github/jalaneunos/neural_network_architectures/blob/main/neuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import os

# Training patterns and labels

In [67]:
X = np.array([[0.09, -0.44, -0.15],
              [0.69, -0.99, -0.76],
              [0.34, 0.65, -0.73],
              [0.15, 0.78, -0.58],
              [-0.63, -0.78, -0.56],
              [0.96, 0.62, -0.66],
              [0.63, -0.45, -0.14],
              [0.88, 0.64, -0.33]])
Y = np.array([-2.57, -2.97, 0.96, 1.04, -3.21, 1.05, -2.39, 0.66])

# Neuron without using PyTorch

In [74]:
class NeuronNP:
  def __init__(self, size, lr=0.01):
    self.w = np.random.rand(size)
    self.b = 0.0
    self.lr = lr

  def relu(self, u):
    return max(0, u)

  def forward(self, x):
    u = np.dot(self.w, x) + self.b
    y = self.relu(u)
    return u, y

  def backpropagation(self, d, y, x):
    derivative_of_sigmoid = 1 if y > 0 else 0
    grad_wrt_w = -(d - y) * derivative_of_sigmoid * x
    grad_wrt_b = -(d - y) * derivative_of_sigmoid

    self.w = self.w - self.lr * grad_wrt_w
    self.b = self.b - self.lr * grad_wrt_b

  def loss(self, d, y):
    return 0.5 * (d - y) ** 2


# Training the handmade perceptron using SGD

In [75]:
neuron_np = NeuronNP(3)

In [76]:
neuron_np.w

array([0.0694203 , 0.44263379, 0.21639073])

In [77]:
NUM_OF_EPOCHS = 200
idx = np.arange(len(X))
for epoch in range(NUM_OF_EPOCHS):
  np.random.shuffle(idx)
  X, Y = X[idx], Y[idx]
  errors = []
  for i in range(len(X)):
    x = X[i]
    d = Y[i]

    u, y = neuron_np.forward(x)
    errors.append(neuron_np.loss(d, y))
    neuron_np.backpropagation(d, y, x)

  if epoch % 10 == 0:
    print(f"Epoch: {epoch}, mean error: {sum(errors) / len(errors)}")
    errors.clear()


Epoch: 0, mean error: 2.0940681238689107
Epoch: 10, mean error: 2.0006753954879692
Epoch: 20, mean error: 1.981413879576471
Epoch: 30, mean error: 1.975565492108188
Epoch: 40, mean error: 1.9730999780436431
Epoch: 50, mean error: 1.9717676497062133
Epoch: 60, mean error: 1.9713489439077851
Epoch: 70, mean error: 1.9710986403016109
Epoch: 80, mean error: 1.970899878770807
Epoch: 90, mean error: 1.9707186571570734
Epoch: 100, mean error: 1.9705511495581352
Epoch: 110, mean error: 1.9703948237360582
Epoch: 120, mean error: 1.970250186339846
Epoch: 130, mean error: 1.9701142955266193
Epoch: 140, mean error: 1.969990353786153
Epoch: 150, mean error: 1.9698724040602973
Epoch: 160, mean error: 1.969762777820144
Epoch: 170, mean error: 1.9696592367182855
Epoch: 180, mean error: 1.9695627243067366
Epoch: 190, mean error: 1.969471437168767


# Neuron with PyTorch

In [99]:
class NeuronPT(nn.Module):
    def __init__(self, input_size):
        super(NeuronPT, self).__init__()
        self.fc = nn.Linear(input_size, 1)  # Single neuron

    def forward(self, x):
        x = self.fc(x)
        x = torch.relu(x)  # ReLU activation function
        return x.squeeze()

# Training PyTorch neuron

In [100]:
neuron = NeuronPT(3)

In [101]:
with torch.no_grad():  # We don't want these actions to be tracked in the computation graph
    neuron.fc.weight = nn.Parameter(torch.tensor([0.77, 0.02, 0.63], dtype=torch.double).view(1, -1))
    neuron.fc.bias = nn.Parameter(torch.tensor([0.0], dtype=torch.double))

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(neuron.parameters(), lr=0.01)


NUM_OF_EPOCHS = 200
for epoch in range(NUM_OF_EPOCHS):

    outputs = neuron(torch.tensor(X, dtype=torch.double))
    loss = criterion(outputs, torch.tensor(Y, dtype=torch.double))


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Loss: {loss.item()}")

Epoch 0/200, Loss: 4.52334044875
Epoch 10/200, Loss: 4.44615263765318
Epoch 20/200, Loss: 4.404935201579156
Epoch 30/200, Loss: 4.366748535301183
Epoch 40/200, Loss: 4.3310902392142445
Epoch 50/200, Loss: 4.297576516021377
Epoch 60/200, Loss: 4.276053372583611
Epoch 70/200, Loss: 4.265445304171971
Epoch 80/200, Loss: 4.256890285088464
Epoch 90/200, Loss: 4.248797024472807
Epoch 100/200, Loss: 4.2426691053004815
Epoch 110/200, Loss: 4.236558982984865
Epoch 120/200, Loss: 4.230609542622343
Epoch 130/200, Loss: 4.226654736645009
Epoch 140/200, Loss: 4.222060199748498
Epoch 150/200, Loss: 4.19755944420016
Epoch 160/200, Loss: 4.150086758825736
Epoch 170/200, Loss: 4.114082748814562
Epoch 180/200, Loss: 4.084068195848394
Epoch 190/200, Loss: 4.062025219780939
